In [76]:
import pandas as pd
from lxml import etree
import requests
import json

---- we can search for a place and then return first hit for a search id ----

In [77]:
#use dict to search for listings
url = "https://www.hemnet.se/locations/show"
params= {
    'q':'östermalm, stockholm'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)



In [78]:
json_data=json.loads(response.text)
search_id=json_data[0]['id']

In [79]:
#or make dict with places to search for

#location_ids={'Ringvägen':475989,'Huvudsta':473498}
#search_id=location_ids['Ringvägen']

In [80]:
#use dict to search for listings
url = "https://www.hemnet.se/bostader"
params= {
    'housing_form_groups':'apartments',
    'location_ids':search_id,
    'item_types':'bostadsratt'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.url

'https://www.hemnet.se/bostader?housing_form_groups=apartments&location_ids=473448&item_types=bostadsratt'

--------- w beautiful soup ------------

In [81]:
from bs4 import BeautifulSoup

In [82]:
soup = BeautifulSoup(response.content, "html.parser")

In [83]:
#find div with good data to use
map_results=soup.find(id='results-map')
initial_data=map_results.attrs['data-initial-data']

In [84]:
#find our search key to use
json_data=json.loads(initial_data)
json_data['search_key']


'7d93d049ab026b63e49ef7fa52e772d753e241e8'

In [85]:
url = "https://www.hemnet.se/bostader/search/"+json_data['search_key']

params= {}

payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)

#print(response.text)

In [86]:
r_json = response.json()
r_prop = r_json['properties']
#r_prop

In [87]:
df=pd.json_normalize(r_prop, max_level=0)
df.head(3)

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,...,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,trackingData,broker_name,broker_phone,broker_email,price_change
0,19226998,"[59.337013065191975, 18.0787619183812]",Nybrogatan 45A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,https://bilder.hemnet.se/images/broker_logo_2/...,Eklund Stockholm New York,"Östermalm, Stockholms kommun",False,False,"{'listingId': '19226998', 'listingType': 'comm...",NaN,NaN,NaN,NaN
1,19227035,"[59.3341255, 18.0876961]",Storgatan 32,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,https://bilder.hemnet.se/images/broker_logo_2/...,Unik Fastighetsförmedling AB,"Östermalm, Stockholms kommun",False,False,"{'listingId': '19227035', 'listingType': 'comm...",NaN,NaN,NaN,NaN
2,19226042,"[59.33749, 18.08601]",Kommendörsgatan 33A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,https://bilder.hemnet.se/images/broker_logo_2/...,Svensk Fastighetsförmedling Sthlm Östermalm/Gä...,"Östermalm, Stockholms kommun",False,False,"{'listingId': '19226042', 'listingType': 'comm...",NaN,NaN,NaN,NaN
